In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pyfasta import Fasta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
plt.rcParams['font.family'] = ['Arial Unicode MS'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号
 
sns.set_style('whitegrid',{'font.sans-serif':['Arial Unicode MS','Arial']})
from statannot import add_stat_annotation

In [ ]:
f = Fasta("/Users/rorschach/genome_ref/hg19.fa")

all_sample=[x for x in os.listdir("./all_ecc_data/raw_data/") if x[-3:]=="txt"]

for file in all_sample:
    temp=pd.read_table("all_ecc_data/raw_data/"+file,index_col=0)
    temp["start_seq"]=temp.apply(lambda x:f.sequence({'chr': x['chr'], 
                                                      'start': x['start']-20, 'stop': x['start']+20}),axis=1)
    temp["end_seq"]=temp.apply(lambda x:f.sequence({'chr': x['chr'], 
                                                      'start': x['end']-20, 'stop': x['end']+20}),axis=1)
    temp.to_csv("all_ecc_data/ecc_data_seq_extend/seq_"+file,sep="\t")
    

In [2]:
import difflib

def compareseq(a,b):
    a=a.upper()
    b=b.upper()
    br=b[::-1]
    alls=difflib.SequenceMatcher(None,a,b).ratio()
    alls_r=difflib.SequenceMatcher(None,a,br).ratio()
    se=difflib.SequenceMatcher(None,a[21:],b[21:]).ratio()
    ser=difflib.SequenceMatcher(None,a[21:],br[:20]).ratio()
    es=difflib.SequenceMatcher(None,a[:20],b[:20]).ratio()
    esr=difflib.SequenceMatcher(None,a[:20],br[21:]).ratio()
    return pd.Series([alls,alls_r,se,ser,es,esr])

In [ ]:
all_sample=[x for x in os.listdir("./all_ecc_data/ecc_data_seq_extend/") if x[-3:]=="txt"]

for file in all_sample:
    temp=pd.read_table("all_ecc_data/ecc_data_seq_extend/"+file,index_col=0)
    temp[["alls","alls_r","se","ser","es","esr"]]=temp.apply(lambda x:compareseq(str(x["start_seq"]),
                                                                            str(x["end_seq"])),axis=1).values

    temp.drop(["start_seq","end_seq"],axis=1).to_csv("./all_ecc_data/ecc_data_seq_sim/sim_"+file,sep="\t")
    